# Assessing the module

Installing [tensorly](http://tensorly.org/)

In [1]:
!pip install -U tensorly

In [118]:
import tensorly as tl
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
from tensorly import decomposition

Define a NumPy array of 4 dimensions, which here represents 4 qubits state.

In [119]:
tensor = tl.tensor([0, 1, 3, 0.05, 0, 2, 6, 0, 0, 1, 1, 0.05, 0, 2, 2, -0.1]).reshape(2, 2, 2, 2)

# be a valid quantum state!
tensor = tensor / sum((tensor.flatten() ** 2)) ** .5
print(tensor)

[[[[0.000 0.129]
   [0.387 0.006]]

  [[0.000 0.258]
   [0.774 0.000]]]


 [[[0.000 0.129]
   [0.129 0.006]]

  [[0.000 0.258]
   [0.258 -0.013]]]]


In [130]:
tt = decomposition.tensor_train(tensor, rank=2)
tt.factors

[array([[[-0.924, -0.383],
         [-0.383, 0.924]]]),
 array([[[0.447, -0.003],
         [0.894, 0.001]],
 
        [[0.000, -0.445],
         [0.000, -0.896]]]),
 array([[[-0.000, 0.924],
         [-0.997, -0.002]],
 
        [[-0.000, 0.383],
         [0.071, -0.022]]]),
 array([[[0.913],
         [0.000]],
 
        [[0.000],
         [-0.408]]])]

How the train looks like:

In [131]:
from functools import reduce

print("Original tensor shape: ", tensor.shape)
print("\nTotal elements: ", reduce(lambda a, b: a * b, tensor.shape, 1))
print("\nTrain length: ", len(tensor.shape))
print("\nTrain cars: ")
size = 0
for f in tt.factors:
    print("\t", f.shape, end="->")
    size += reduce(lambda a, b: a * b, f.shape, 1)
print("\n\nTotal train size: ", size)

Original tensor shape:  (2, 2, 2, 2)

Total elements:  16

Train length:  4

Train cars: 
	 (1, 2, 2)->	 (2, 2, 2)->	 (2, 2, 2)->	 (2, 2, 1)->

Total train size:  24


How to restore a tensor

In [132]:
print(tt.to_tensor())

[[[[0.000 0.129]
   [0.387 0.001]]

  [[0.000 0.258]
   [0.774 0.003]]]


 [[[-0.000 0.129]
   [0.129 -0.004]]

  [[-0.000 0.258]
   [0.258 -0.008]]]]


In [133]:
print("Allclose: ", np.allclose(tt.to_tensor(), tensor, atol=1e-5))
print("Error: ", np.linalg.norm(tensor - tt.to_tensor()))

Allclose:  False
Error:  0.012889545659941715


Restore the tensor with our hands!

In [140]:
t_core_copy = t_core.reshape(2, 2)

# along the mode 3 we keep the size
t_core_copy = (t_matrices[3] @ t_core_copy.reshape(2, 2).T).T.reshape((1, 1, 2, 2))

# along the mode 2 we keep the size
t_core_copy = (t_matrices[2] @ t_core_copy.reshape(2, 2)).reshape((1, 1, 2, 2))

# along the mode 1 we make size from 1 to 2
t_core_copy = np.kron(t_matrices[1], t_core_copy.reshape(4, 1)).reshape((1, 2, 2, 2))

# along the mode 0 we make size from 1 to 2
t_core_copy = np.kron(t_matrices[0], t_core_copy.reshape(8, 1)).reshape((2, 2, 2, 2))
t_core_copy

array([[[[0.000, 0.156],
         [0.376, -0.000]],

        [[0.000, 0.312],
         [0.752, -0.001]]],


       [[[0.000, 0.065],
         [0.156, -0.000]],

        [[0.000, 0.129],
         [0.312, -0.000]]]])

Did we do it right?

In [142]:
np.allclose(t_core_copy, tl.tucker_tensor.tucker_to_tensor((t_core, t_matrices)))

True